#  TP 2: 

### 18 mars 2024

### Par Samuel Fortin, Philippe Truchon et Benjamin Trudel

## TP2.1 Décomposition QR par la méthode Householder

#### Fonctions générales

In [239]:
import fnmatch
import functools
import os

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import math


sns.set_theme(style="ticks", palette="deep")

plt.rcParams["axes.spines.right"] = False
plt.rcParams["axes.spines.top"] = False



# Liste les noms de fichier d'un dossier
def listNameOfFiles(directory: str, extension="csv"):
    found_files = []
    for file in os.listdir(directory):
        if fnmatch.fnmatch(file, f"*.{extension}"):
            found_files.append(file)
    return found_files


# Lis et crée une matrice numpy à partir de chemin d'un fichier texte
def readTXT(path: str):
    fich = open(path, "r")
    fich_str = list(fich)
    fich.close()
    x = []
    for i in fich_str:
        elem_str = i.replace("\n", "")
        x.append(float(elem_str))
    return np.array(x)

sign = lambda x: math.copysign(1, x) # two will work

path = os.path.abspath("")
files_name = listNameOfFiles(path)


### Questions:

### a)

<i>À l’aide des équations (2.1.2) et (2.1.3), démontrez que les matrices de réflexion Q_i sont orthogonales.</i>

Sachant que $v^T v = I$:

\begin{gather*}
 H_{m,i}H_{m,i}^T = H_{m,i}H_{m,i}
 \\
 H_{m,i}H_{m,i}^T = \Big( I - \frac{2 v v^T}{v^T v} \Big) \Big( I - \frac{2 v v^T}{v^T v} \Big)
 \\
 H_{m,i}H_{m,i}^T = I - \frac{ 4 v v^T }{ v^T v } + \frac{ 4 v (v^T v) v^T }{ (v^T v)^2 }
 \\
 H_{m,i}H_{m,i}^T = I - \frac{ 4 v v^T }{ v^T v } + \frac{ 4 v v^T }{ v^T v }
 \\
 H_{m,i}H_{m,i}^T = I
\end{gather*}  


$$ Q_i = \left [
\begin{matrix}
I_i & 0 \\
0 & H_{m,i} 
\end{matrix}
\right ]$$

En utilisant la forme générale du produit des matrices par blocs, on sait que:

$$ M^T = \left [
\begin{matrix}
A^T & B^T \\
C^T & D^T 
\end{matrix}
\right ] $$

Donc:

$$ Q_i = \left [
\begin{matrix}
I_i & 0 \\
0 & H_{m,i} 
\end{matrix}
\right ] = \left [
\begin{matrix}
I_i^T & 0 \\
0 & H_{m,i}^T 
\end{matrix}
\right ] = Q_i^T$$





### b)

<i>Démontrez l’équation (2.1.5) et que la matrice Q est orthogonale.</i>

 

En utilisant la propriété d'associativité : $(AB)^T = B^T A^T$ et sachant que $Q_{i}^T Q_{i} = I$

\begin{gather*}
    Q = \prod_{i=0}^{n-1} Q_i^T
\end{gather*}

\begin{gather*}
    (\prod_{i=0}^{n-1} Q_i)^T (\prod_{i=0}^{n-1} Q_i) = (\prod_{i=n-1}^{0} Q_i^T)(\prod_{i=0}^{n-1} Q_i) = \prod_{i=0}^{n-1} I = I
\end{gather*}

La matrice Q est donc orthogonale.

### c)

<i>Implémentez la fonction householder_qr qui prend en argument une matrice A et qui retourne les matrices Q
et R obtenues par la méthode de Householder.</i>


In [240]:
def householder_qr(A, inter=False):
    m, n = A.shape
    Q = np.identity(m)
    for i in range(n):
        H = np.identity(m)
        x = A[i:, i]

        e1 = np.zeros(len(x)).T
        e1[0] = 1

        v = np.array([x]).T+np.copysign(np.linalg.norm(x), x[0]) * np.array([e1]).T


        Ht = np.identity(x.shape[0])

        Ht -= (2 * (v @ v.T)/(v.T @ v))

        H[i:, i:] = Ht
        Q = Q @ H
        A = H @ A
        if inter:
            print(f'A{i}:\n', A.round(6)) 
    return Q, A

### d)

<i>À l’aide d’une matrice de dimension 4 × 3 de votre choix, testez votre fonction householder_qr et comparez les
résultats obtenus avec ceux obtenus à l’aide de la fonction numpy.linalg.qr. Les matrices sont-elles exactement
les mêmes ? Si non, est-ce un problème?</i>



In [241]:
a = np.random.randint(1,10,(4,3))
print(a)
Q, R = householder_qr(a)
print('Q:\n', Q.round(6))
print('R:\n', R.round(6))
r2 = np.linalg.qr(a, mode='complete')
print(r2)

[[7 5 3]
 [6 3 8]
 [6 1 4]
 [3 1 5]]
Q:
 [[-0.613941  0.655208 -0.403697  0.175524]
 [-0.526235  0.054601  0.653604 -0.541199]
 [-0.526235 -0.734075 -0.422921 -0.073135]
 [-0.263117 -0.169869  0.480592  0.819113]]
R:
 [[-11.401754  -5.43776   -9.472227]
 [  0.         2.535896  -1.383216]
 [  0.         0.         4.729021]
 [ -0.        -0.        -0.      ]]
QRResult(Q=array([[-0.61394061,  0.65520762, -0.40369689,  0.17552416],
       [-0.52623481,  0.05460063,  0.65360448, -0.54119948],
       [-0.52623481, -0.7340752 , -0.42292055, -0.07313506],
       [-0.26311741, -0.16986864,  0.48059153,  0.81911273]]), R=array([[-11.40175425,  -5.43775972,  -9.47222661],
       [  0.        ,   2.53589614,  -1.38321608],
       [  0.        ,   0.        ,   4.72902066],
       [  0.        ,   0.        ,   0.        ]]))


Les matrices sont les mêmes, mais la matrice Q pourrait avoir des signes différents selon la convention de signe utilisée qui menerait à la même matrice R.

### e)

<i>À l’aide de la matrice utilisée en d, illustrez comment la multiplication successive des matrices Q_i triangularise
progressivement la matrice A. Dans l’élan, assurez-vous que les matrices Q et R obtenues sont bien orthogonale et
triangulaire supérieure, respectivement.</i>


In [242]:
Q, R = householder_qr(a, inter=True)
I = Q @ Q.T
print(I)

A0:
 [[-11.401754  -5.43776   -9.472227]
 [  0.        -0.403293   3.933357]
 [ -0.        -2.403293  -0.066643]
 [ -0.        -0.701646   2.966679]]
A1:
 [[-11.401754  -5.43776   -9.472227]
 [  0.         2.535896  -1.383216]
 [ -0.        -0.        -4.413856]
 [ -0.        -0.         1.6975  ]]
A2:
 [[-11.401754  -5.43776   -9.472227]
 [  0.         2.535896  -1.383216]
 [  0.         0.         4.729021]
 [ -0.        -0.        -0.      ]]
[[ 1.00000000e+00  1.12650624e-16  1.49590087e-17  9.33017988e-18]
 [ 1.12650624e-16  1.00000000e+00  6.91451013e-18 -6.09758937e-17]
 [ 1.49590087e-17  6.91451013e-18  1.00000000e+00  2.95043675e-17]
 [ 9.33017988e-18 -6.09758937e-17  2.95043675e-17  1.00000000e+00]]


On oberve qu'à chaque itération la colonne la plus à gauche non triangulaire le devient en devenant des zéros en dessous de la diagonale. En même temps, on observe que la dernière matrice A qui est égale à R est triangulaire supérieur que tous les valeurs en dessous de la diagonale sont zéros. Finalement, on vérifie l'orthogonalité de Q en multipliant Q par sa transpose. Le résultat obtenu est la matrice identité comme prévu, confirmant l'orthogonalité. À noter, que les zéros dans ce cas ne sont pas exactement zéro à cause d'erreur numérique.

In [243]:
I[I< 1e-15] = 0
print(I)

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
